<a href="https://colab.research.google.com/github/Nuclei-7/FirstMachineLearningModel/blob/main/imageCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install Dependencies and Setup

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip list

In [25]:
import tensorflow as tf
import os

In [26]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

# 2. Remove dodgy images

In [ ]:
!git clone https://github.com/ThunderE75/CNN-ImageClassification.git

In [29]:
data_dir = '/content/CNN/data'

In [30]:
import os
import cv2
import imghdr

# Set data directory
data_dir = '/content/CNN/data'

# Supported image extensions
image_exts = ['jpeg', 'jpg', 'png', 'bmp']

# Iterate over the classes (folders) and images
for image_class in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, image_class)

    # Check if it's a directory
    if os.path.isdir(class_dir):
        for image in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image)
            try:
                # Read image and check extension
                img = cv2.imread(image_path)
                tip = imghdr.what(image_path)

                # Check if the file has a valid image extension
                if tip not in image_exts:
                    print('Image not in ext list {}'.format(image_path))
                    os.remove(image_path)
            except Exception as e:
                print('Issue with image {}'.format(image_path))
                # os.remove(image_path)  # Uncomment if you want to delete corrupted files


# 3. Load Data

In [31]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [34]:
data_iterator = data.as_numpy_iterator()

In [35]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [37]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

# 5. Split Data

In [39]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [41]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

# 6. Build Deep Learning

In [ ]:
train

In [43]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [44]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [46]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Training

In [48]:
logdir='logs'

In [49]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# 8. plot performence

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='red', label='loss')
plt.plot(hist.history['val_loss'], color='blue', label='val_loss')
fig.suptitle('Loss', fontsize=18)
plt.legend(loc="upper rigt")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='green', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluation

In [54]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [55]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

# 10. Testing

In [ ]:
import cv2
import requests
import numpy as np
import matplotlib.pyplot as plt

# URL of the raw image from GitHub

url = "https://github.com/ThunderE75/CNN-ImageClassification/blob/2699669029524d58a111c7918b8fb7a2d8f7a1f1/test_image1.jpg"

# Download the image
response = requests.get(url)

if response.status_code == 200:  # Check if the request was successful
    image_array = np.frombuffer(response.content, np.uint8)
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Display the image
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
    plt.axis('off')  # Hide the axes
    plt.show()
else:
    print("Error: Unable to download the image.")


In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5:
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

# 11. Save the Model

In [73]:
from tensorflow.keras.models import load_model

In [80]:
model.save(os.path.join('models','imageclassifier.keras'))

In [ ]:
import os
import numpy as np
from keras.models import load_model

# Assuming the model is already loaded into a variable called `model`
# Example code to load the model
model_path = 'models/imageclassifier.keras'  # Adjust if necessary
if os.path.isfile(model_path):
    model = load_model(model_path)
    print("Model loaded successfully!")
else:
    print(f"Error: The file '{model_path}' does not exist.")

# Assuming you have already resized the image (and it's stored in a variable called `resize`)
# Make sure your image is preprocessed correctly (normalized, resized, etc.)
# Example of predicting with the model
prediction = model.predict(np.expand_dims(resize / 255.0, axis=0))

# Print the prediction result
print("Prediction:", prediction)